## Lineær og Polynomisk Regresjon for Prissetting av Gresskar - Leksjon 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografikk av Dasani Madipalli</figcaption>


#### Introduksjon

Så langt har du utforsket hva regresjon er ved hjelp av eksempeldata hentet fra gresskarprissettingsdatasettet som vi vil bruke gjennom denne leksjonen. Du har også visualisert det ved hjelp av `ggplot2`.💪

Nå er du klar til å dykke dypere inn i regresjon for maskinlæring. I denne leksjonen vil du lære mer om to typer regresjon: *grunnleggende lineær regresjon* og *polynomisk regresjon*, sammen med noe av matematikken som ligger til grunn for disse teknikkene.

> Gjennom hele dette kurset antar vi minimal kunnskap om matematikk, og vi søker å gjøre det tilgjengelig for studenter fra andre felt, så se etter notater, 🧮 utrop, diagrammer og andre læringsverktøy som kan hjelpe med forståelsen.

#### Forberedelse

Som en påminnelse, du laster inn disse dataene for å stille spørsmål til dem.

-   Når er det beste tidspunktet å kjøpe gresskar?

-   Hva kan jeg forvente å betale for en kasse med miniatyrgresskar?

-   Bør jeg kjøpe dem i halv-bushel kurver eller i 1 1/9 bushel bokser? La oss fortsette å grave i disse dataene.

I forrige leksjon opprettet du en `tibble` (en moderne nytenkning av dataframes) og fylte den med en del av det opprinnelige datasettet, standardiserte prissettingen etter bushel. Ved å gjøre det, var du imidlertid bare i stand til å samle rundt 400 datapunkter og kun for høstmånedene. Kanskje vi kan få litt mer detaljert informasjon om naturen til dataene ved å rense dem mer? Vi får se... 🕵️‍♀️

For denne oppgaven trenger vi følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling av R-pakker](https://www.tidyverse.org/packages) designet for å gjøre dataanalyse raskere, enklere og morsommere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) rammeverket er en [samling av pakker](https://www.tidymodels.org/packages/) for modellering og maskinlæring.

-   `janitor`: [janitor-pakken](https://github.com/sfirke/janitor) gir enkle verktøy for å undersøke og rense skitne data.

-   `corrplot`: [corrplot-pakken](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) gir et visuelt utforskingsverktøy for korrelasjonsmatriser som støtter automatisk variabelreorganisering for å hjelpe med å oppdage skjulte mønstre blant variabler.

Du kan installere dem slik:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Skriptet nedenfor sjekker om du har de nødvendige pakkene for å fullføre dette modulen og installerer dem for deg hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Vi vil senere laste inn disse fantastiske pakkene og gjøre dem tilgjengelige i vår nåværende R-sesjon. (Dette er kun for illustrasjon, `pacman::p_load()` har allerede gjort dette for deg)

## 1. En lineær regresjonslinje

Som du lærte i leksjon 1, er målet med en lineær regresjonsøvelse å kunne tegne en *linje* *for* *beste tilpasning* for å:

-   **Vise variabelsammenhenger**. Vise sammenhengen mellom variabler.

-   **Gjøre prediksjoner**. Gjøre nøyaktige prediksjoner om hvor et nytt datapunkt vil falle i forhold til den linjen.

For å tegne denne typen linje bruker vi en statistisk teknikk kalt **Minste kvadraters regresjon**. Begrepet `minste kvadrater` betyr at alle datapunktene rundt regresjonslinjen kvadreres og deretter summeres. Ideelt sett er den endelige summen så liten som mulig, fordi vi ønsker et lavt antall feil, eller `minste kvadrater`. Som sådan er linjen for beste tilpasning den linjen som gir oss den laveste verdien for summen av de kvadrerte feilene - derav navnet *minste kvadraters regresjon*.

Vi gjør dette fordi vi ønsker å modellere en linje som har den minste kumulative avstanden fra alle våre datapunkter. Vi kvadrerer også termene før vi summerer dem, siden vi er opptatt av størrelsen snarere enn retningen.

> **🧮 Vis meg matematikken**
>
> Denne linjen, kalt *linjen for beste tilpasning*, kan uttrykkes med [en ligning](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` er '`forklaringsvariabelen` eller `prediktoren`'. `Y` er '`avhengig variabel` eller `utfallet`'. Stigningen på linjen er `b`, og `a` er skjæringspunktet med y-aksen, som refererer til verdien av `Y` når `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "stigning = $y/x$")
    Infografikk av Jen Looper
>
> Først, beregn stigningen `b`.
>
> Med andre ord, og med henvisning til vårt opprinnelige spørsmål om gresskar-data: "forutsi prisen på et gresskar per skjeppe etter måned", vil `X` referere til prisen og `Y` referere til salgsdatoen.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografikk av Jen Looper
> 
> Beregn verdien av Y. Hvis du betaler rundt \$4, må det være april!
>
> Matematikk som beregner linjen må demonstrere stigningen på linjen, som også avhenger av skjæringspunktet, eller hvor `Y` befinner seg når `X = 0`.
>
> Du kan se metoden for beregning av disse verdiene på nettstedet [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Besøk også [denne kalkulatoren for minste kvadrater](https://www.mathsisfun.com/data/least-squares-calculator.html) for å se hvordan verdiene påvirker linjen.

Ikke så skummelt, sant? 🤓

#### Korrelasjon

Et annet begrep å forstå er **Korrelasjonskoeffisienten** mellom gitte X- og Y-variabler. Ved hjelp av et spredningsdiagram kan du raskt visualisere denne koeffisienten. Et diagram med datapunkter som er pent spredt i en linje har høy korrelasjon, mens et diagram med datapunkter spredt overalt mellom X og Y har lav korrelasjon.

En god lineær regresjonsmodell vil være en som har en høy (nærmere 1 enn 0) korrelasjonskoeffisient ved bruk av metoden for minste kvadraters regresjon med en regresjonslinje.


## **2. En dans med data: opprette en data frame som skal brukes til modellering**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>


<!--![Kunstverk av \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Last inn nødvendige biblioteker og datasett. Konverter dataene til en data frame som inneholder et utvalg av dataene:

-   Ta kun med gresskar som er priset per skjeppe

-   Konverter datoen til en måned

-   Beregn prisen som et gjennomsnitt av høy og lav pris

-   Konverter prisen slik at den reflekterer prisingen per skjeppekvantitet

> Vi dekket disse stegene i [forrige leksjon](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

I eventyrets ånd, la oss utforske [`janitor-pakken`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) som tilbyr enkle funksjoner for å undersøke og rense uryddige data. For eksempel, la oss se på kolonnenavnene for dataene våre:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Vi kan gjøre det bedre. La oss gjøre disse kolonnenavnene `friendR` ved å konvertere dem til [snake_case](https://en.wikipedia.org/wiki/Snake_case)-konvensjonen ved hjelp av `janitor::clean_names`. For å finne ut mer om denne funksjonen: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Mye ryddigere 🧹! Nå, en dans med dataene ved hjelp av `dplyr`, som i forrige leksjon! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Bra jobba!👌 Du har nå et rent og ryddig datasett som du kan bruke til å bygge din nye regresjonsmodell!

Hva med et spredningsdiagram?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Et spredningsdiagram minner oss om at vi kun har månedsdata fra august til desember. Vi trenger sannsynligvis mer data for å kunne trekke konklusjoner på en lineær måte.

La oss ta en titt på modelldataene våre igjen:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Hva om vi ønsket å forutsi `price` på et gresskar basert på kolonnene `city` eller `package`, som er av typen karakter? Eller enda enklere, hvordan kunne vi finne korrelasjonen (som krever at begge inputene er numeriske) mellom for eksempel `package` og `price`? 🤷🤷

Maskinlæringsmodeller fungerer best med numeriske egenskaper i stedet for tekstverdier, så du må vanligvis konvertere kategoriske egenskaper til numeriske representasjoner.

Dette betyr at vi må finne en måte å omforme våre prediktorer på for å gjøre dem enklere for en modell å bruke effektivt, en prosess kjent som `feature engineering`.


## 3. Forbehandling av data for modellering med oppskrifter 👩‍🍳👨‍🍳

Aktiviteter som omformaterer prediktorverdier for å gjøre dem enklere for en modell å bruke effektivt, har blitt kalt `feature engineering`.

Ulike modeller har ulike krav til forbehandling. For eksempel krever minste kvadraters metode `koding av kategoriske variabler` som måned, sort og bynavn. Dette innebærer ganske enkelt å `oversette` en kolonne med `kategoriske verdier` til en eller flere `numeriske kolonner` som erstatter den opprinnelige.

For eksempel, anta at dataene dine inkluderer følgende kategoriske variabel:

|   by    |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

Du kan bruke *ordinal koding* for å erstatte hver kategori med en unik heltallsverdi, slik som dette:

| by  |
|:---:|
|  0  |
|  1  |
|  2  |

Og det er akkurat det vi skal gjøre med dataene våre!

I denne delen skal vi utforske et annet fantastisk Tidymodels-pakke: [recipes](https://tidymodels.github.io/recipes/) - som er designet for å hjelpe deg med å forbehandle dataene dine **før** du trener modellen din. I kjernen er en oppskrift et objekt som definerer hvilke steg som skal brukes på et datasett for å gjøre det klart for modellering.

La oss nå lage en oppskrift som forbereder dataene våre for modellering ved å erstatte en unik heltallsverdi for alle observasjonene i prediktorkolonnene:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Fantastisk! 👏 Vi har nettopp laget vår første oppskrift som spesifiserer et utfall (pris) og tilhørende prediktorer, og som sørger for at alle prediktorkolonnene blir kodet som et sett med heltall 🙌! La oss raskt bryte det ned:

-   Kallet til `recipe()` med en formel forteller oppskriften *rollene* til variablene ved å bruke `new_pumpkins`-dataene som referanse. For eksempel har `price`-kolonnen blitt tildelt rollen som `outcome`, mens resten av kolonnene har blitt tildelt rollen som `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` spesifiserer at alle prediktorene skal konverteres til et sett med heltall, der nummereringen starter på 0.

Vi er sikre på at du kanskje tenker noe sånt som: "Dette er så kult!! Men hva om jeg trenger å bekrefte at oppskriftene gjør akkurat det jeg forventer? 🤔"

Det er en fantastisk tanke! Du skjønner, når oppskriften din er definert, kan du estimere parameterne som kreves for faktisk å forhåndsbehandle dataene, og deretter trekke ut de bearbeidede dataene. Du trenger vanligvis ikke å gjøre dette når du bruker Tidymodels (vi skal snart se den normale konvensjonen -\> `workflows`), men det kan være nyttig når du vil gjøre en slags kontroll for å bekrefte at oppskriftene gjør det du forventer.

For dette trenger du to flere verb: `prep()` og `bake()`, og som alltid hjelper våre små R-venner fra [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) deg med å forstå dette bedre!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): estimerer de nødvendige parameterne fra et treningssett som senere kan brukes på andre datasett. For eksempel, for en gitt prediktorkolonne, hvilken observasjon vil bli tildelt heltall 0, 1, 2 osv.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): tar en ferdigbehandlet oppskrift og utfører operasjonene på et hvilket som helst datasett.

Med det sagt, la oss forberede og bruke oppskriftene våre for virkelig å bekrefte at prediktorkolonnene under panseret først vil bli kodet før en modell tilpasses.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 De bearbeidede dataene `baked_pumpkins` har alle sine prediktorer kodet, noe som bekrefter at forhåndsprosesseringsstegene definert som vår oppskrift fungerer som forventet. Dette gjør det vanskeligere for deg å lese, men mye mer forståelig for Tidymodels! Ta deg tid til å finne ut hvilken observasjon som har blitt tilordnet et tilsvarende heltall.

Det er også verdt å nevne at `baked_pumpkins` er en data frame som vi kan utføre beregninger på.

For eksempel, la oss prøve å finne en god korrelasjon mellom to punkter i dataene dine for potensielt å bygge en god prediktiv modell. Vi bruker funksjonen `cor()` for å gjøre dette. Skriv `?cor()` for å finne ut mer om funksjonen.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Som det viser seg, er det bare en svak korrelasjon mellom By og Pris. Men det er en litt bedre korrelasjon mellom Pakke og dens Pris. Det gir mening, ikke sant? Vanligvis, jo større produktboksen er, desto høyere er prisen.

Mens vi er i gang, la oss også prøve å visualisere en korrelasjonsmatrise for alle kolonnene ved hjelp av `corrplot`-pakken.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Mye bedre.

Et godt spørsmål å stille nå vil være: '`Hvilken pris kan jeg forvente for en gitt gresskarpakke?`' La oss sette i gang!

> Merk: Når du **`bake()`** den forberedte oppskriften **`pumpkins_prep`** med **`new_data = NULL`**, henter du ut de bearbeidede (dvs. kodede) treningsdataene. Hvis du hadde et annet datasett, for eksempel et testsett, og ønsket å se hvordan en oppskrift ville forhåndsbehandle det, ville du ganske enkelt bake **`pumpkins_prep`** med **`new_data = test_set`**

## 4. Bygg en lineær regresjonsmodell

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografikk av Dasani Madipalli</figcaption>


<!--![Infografikk av Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Nå som vi har laget en oppskrift og faktisk bekreftet at dataene vil bli forhåndsbehandlet riktig, la oss nå bygge en regresjonsmodell for å svare på spørsmålet: `Hvilken pris kan jeg forvente for en gitt gresskarpakke?`

#### Tren en lineær regresjonsmodell ved hjelp av treningssettet

Som du kanskje allerede har forstått, er kolonnen *price* den `avhengige` variabelen, mens kolonnen *package* er den `uavhengige` variabelen.

For å gjøre dette, vil vi først dele opp dataene slik at 80% går til treningssettet og 20% til testsettet, deretter definere en oppskrift som vil kode den uavhengige variabelen til et sett med heltall, og deretter bygge en modellspesifikasjon. Vi vil ikke forberede og bake oppskriften vår siden vi allerede vet at den vil forhåndsbehandle dataene som forventet.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Bra jobbet! Nå som vi har en oppskrift og en modellspesifikasjon, må vi finne en måte å samle dem sammen i et objekt som først vil forhåndsbehandle dataene (prep+bake i bakgrunnen), tilpasse modellen til de forhåndsbehandlede dataene, og også tillate mulige etterbehandlingsaktiviteter. Hvordan er det for din sinnsro!🤩

I Tidymodels kalles dette praktiske objektet en [`workflow`](https://workflows.tidymodels.org/) og holder praktisk dine modelleringskomponenter! Dette er det vi ville kalt *pipelines* i *Python*.

Så la oss samle alt i en workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

I tillegg kan en arbeidsflyt tilpasses/trenes på samme måte som en modell kan.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Fra modellens output kan vi se koeffisientene som ble lært under treningen. De representerer koeffisientene for den beste tilpassede linjen som gir oss den laveste totale feilen mellom den faktiske og den predikerte variabelen.

#### Evaluere modellens ytelse ved bruk av testsettet

Det er på tide å se hvordan modellen presterte 📏! Hvordan gjør vi dette?

Nå som vi har trent modellen, kan vi bruke den til å lage prediksjoner for test_set ved hjelp av `parsnip::predict()`. Deretter kan vi sammenligne disse prediksjonene med de faktiske verdiene for å evaluere hvor godt (eller dårlig!) modellen fungerer.

La oss starte med å lage prediksjoner for testsettet og deretter binde kolonnene til testsettet.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Ja, du har nettopp trent en modell og brukt den til å gjøre prediksjoner! 🔮 Er den noe bra? La oss evaluere modellens ytelse!

I Tidymodels gjør vi dette ved hjelp av `yardstick::metrics()`! For lineær regresjon, la oss fokusere på følgende metrikker:

-   `Root Mean Square Error (RMSE)`: Kvadratroten av [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Dette gir en absolutt metrikk i samme enhet som målet (i dette tilfellet prisen på et gresskar). Jo mindre verdien er, desto bedre er modellen (enkelt sagt representerer det gjennomsnittsprisen modellen tar feil med!).

-   `Coefficient of Determination (vanligvis kjent som R-squared eller R2)`: En relativ metrikk der høyere verdi betyr bedre tilpasning av modellen. I hovedsak representerer denne metrikken hvor mye av variasjonen mellom de predikerte og faktiske måleverdiene modellen klarer å forklare.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Der går modellens ytelse. La oss se om vi kan få en bedre indikasjon ved å visualisere et spredningsdiagram av pakken og prisen, og deretter bruke de prediksjonene som er gjort til å legge over en linje for beste tilpasning.

Dette betyr at vi må forberede og bearbeide testsettet for å kode pakke-kolonnen, og deretter binde dette til prediksjonene som er gjort av modellen vår.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Flott! Som du kan se, generaliserer ikke lineær regresjonsmodellen særlig godt forholdet mellom en pakke og dens tilsvarende pris.

🎃 Gratulerer, du har nettopp laget en modell som kan hjelpe med å forutsi prisen på noen varianter av gresskar. Din gresskarhage til høytiden kommer til å bli fantastisk. Men du kan sannsynligvis lage en bedre modell!

## 5. Bygg en polynomisk regresjonsmodell

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografikk av Dasani Madipalli</figcaption>


<!--![Infografikk av Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Noen ganger har ikke dataene våre en lineær sammenheng, men vi ønsker likevel å forutsi et utfall. Polynomisk regresjon kan hjelpe oss med å lage prediksjoner for mer komplekse ikke-lineære sammenhenger.

Ta for eksempel forholdet mellom pakke og pris i vårt gresskar-datasett. Selv om det noen ganger er en lineær sammenheng mellom variabler – jo større gresskaret er i volum, desto høyere pris – kan det hende at disse sammenhengene ikke kan plottes som et plan eller en rett linje.

> ✅ Her er [noen flere eksempler](https://online.stat.psu.edu/stat501/lesson/9/9.8) på data som kan bruke polynomisk regresjon
>
> Ta en ny titt på forholdet mellom Sort til Pris i det forrige plottet. Virker det som om dette spredningsdiagrammet nødvendigvis bør analyseres med en rett linje? Kanskje ikke. I dette tilfellet kan du prøve polynomisk regresjon.
>
> ✅ Polynom er matematiske uttrykk som kan bestå av én eller flere variabler og koeffisienter

#### Tren en polynomisk regresjonsmodell ved hjelp av treningssettet

Polynomisk regresjon lager en *kurvet linje* for bedre å tilpasse ikke-lineære data.

La oss se om en polynomisk modell vil prestere bedre i å lage prediksjoner. Vi følger en prosedyre som ligner på den vi brukte tidligere:

-   Lag en oppskrift som spesifiserer forbehandlingsstegene som skal utføres på dataene våre for å gjøre dem klare for modellering, dvs.: koding av prediktorer og beregning av polynomer av grad *n*

-   Bygg en modellspecifikasjon

-   Kombiner oppskriften og modellspecifikasjonen i en arbeidsflyt

-   Lag en modell ved å tilpasse arbeidsflyten

-   Evaluer hvor godt modellen presterer på testdataene

La oss sette i gang!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Evaluere modellens ytelse

👏👏Du har laget en polynommodell, la oss gjøre spådommer på testsettet!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, la oss evaluere hvordan modellen presterte på test_set ved å bruke `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Mye bedre ytelse.

`rmse` gikk ned fra omtrent 7 til omtrent 3, noe som indikerer en redusert feil mellom den faktiske prisen og den predikerte prisen. Du kan *løst* tolke dette som at feilaktige prediksjoner i gjennomsnitt er feil med rundt 3 dollar. `rsq` økte fra omtrent 0,4 til 0,8.

Alle disse målene viser at den polynomiske modellen fungerer mye bedre enn den lineære modellen. Bra jobbet!

La oss se om vi kan visualisere dette!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Du kan se en kurvet linje som passer bedre til dataene dine! 🤩

Du kan gjøre dette enda jevnere ved å sende en polynomformel til `geom_smooth` slik som dette:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Akkurat som en jevn kurve!🤩

Slik kan du lage en ny prediksjon:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Prediksjonen fra `polynomial model` gir mening, gitt spredningsdiagrammene for `price` og `package`! Og hvis dette er en bedre modell enn den forrige, basert på de samme dataene, må du planlegge for disse dyrere gresskarene!

🏆 Bra jobbet! Du har laget to regresjonsmodeller i én leksjon. I den siste delen om regresjon vil du lære om logistisk regresjon for å bestemme kategorier.

## **🚀Utfordring**

Test flere forskjellige variabler i denne notatboken for å se hvordan korrelasjon samsvarer med modellens nøyaktighet.

## [**Quiz etter leksjonen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Gjennomgang & Selvstudium**

I denne leksjonen lærte vi om Lineær Regresjon. Det finnes andre viktige typer regresjon. Les om Stepwise, Ridge, Lasso og Elasticnet-teknikker. Et godt kurs for å lære mer er [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Hvis du vil lære mer om hvordan du bruker det fantastiske Tidymodels-rammeverket, kan du sjekke ut følgende ressurser:

-   Tidymodels nettsted: [Kom i gang med Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn og Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **TAKK TIL:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) for å lage de fantastiske illustrasjonene som gjør R mer innbydende og engasjerende. Finn flere illustrasjoner i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
